# Code demonstrating basic non-linear reduced rank models

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import copy

import matplotlib.pyplot as plt
import numpy as np
import numpy.linalg
import torch

from janelia_core.ml.reduced_rank_models import RRReluModel

## Setup ground truth model

In [3]:
d_in, d_out, d_latent, n_smps = 30000, 30000, 10, 10000

In [4]:
true_model = RRReluModel(d_in, d_out, d_latent)
true_model.generate_random_model(var_range=[.1, 1])

## Generate data

In [5]:
x = 3*torch.randn([n_smps, d_in])
y = true_model.generate(x)

## Create model we will fit to data

In [6]:
fitted_model = RRReluModel(d_in, d_out, d_latent)
fitted_model.init_weights(y, w_gain=.0001)
fitted_model.to('cuda')

RRReluModel()

## Fit model to data

In [7]:
adam_params = {'lr': .001, 'betas': [.9, .999]} # betas are for mean and second moment (.9, .999)
fitted_model.fit(x,y, batch_size=100, send_size=100,
                 max_its=30000, update_int=1000, adam_params=adam_params)

0: Elapsed time 0.42189788818359375, vl: 107877.61
1000: Elapsed time 93.51170349121094, vl: 47366.73
2000: Elapsed time 195.86776328086853, vl: 45142.1
3000: Elapsed time 286.578360080719, vl: 40423.895
4000: Elapsed time 377.4561915397644, vl: 38186.0425
5000: Elapsed time 475.69967341423035, vl: 36334.21
6000: Elapsed time 566.3487257957458, vl: 34284.755
7000: Elapsed time 657.4675290584564, vl: 32155.645
8000: Elapsed time 761.5442547798157, vl: 30096.44
9000: Elapsed time 852.1900060176849, vl: 27812.565
10000: Elapsed time 942.9223656654358, vl: 25627.095
11000: Elapsed time 1033.670084953308, vl: 23654.65
12000: Elapsed time 1124.2460460662842, vl: 21450.01
13000: Elapsed time 1214.8382019996643, vl: 19615.86875
14000: Elapsed time 1305.4274833202362, vl: 17586.89
15000: Elapsed time 1408.9329340457916, vl: 16296.1475
16000: Elapsed time 1499.9409878253937, vl: 14845.98625
17000: Elapsed time 1592.6779382228851, vl: 13609.6925
18000: Elapsed time 1695.458619594574, vl: 12647.10

## See fitted model results

In [20]:
%matplotlib qt
true_model.standardize()
fitted_model.standardize()
fitted_model.to('cpu')
RRReluModel.compare_models(true_model, fitted_model, x[0:100,:], [10, 100, 1000])
fitted_model.to('cuda')

Flipping signs for dimension 7
Flipping signs for dimension 8


RRReluModel()

In [14]:
w0_true = true_model.w0.cpu().detach().numpy()
w1_true = true_model.w1.cpu().detach().numpy()

w0_fitted = fitted_model.w0.cpu().detach().numpy()
w1_fitted = fitted_model.w1.cpu().detach().numpy()

w_true = np.matmul(w1_true, w0_true.T)
w_fitted = np.matmul(w1_fitted, w0_fitted.T)

In [15]:
from janelia_core.visualization.matrix_visualization import cmp_n_mats

In [18]:
cmp_n_mats([w_true[0:100, 0:100], w_fitted[0:100, 0:100], w_true[0:100, 0:100]-w_fitted[0:100, 0:100]])